# Install and import libraries

In [84]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
from IPython.display import clear_output, display
import time

In [4]:
# Mount Drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create folium map display function

In [83]:
def create_folium_map(coord, service_stations = None, parking = None, zoom_start = 18, max_zoom = 23):
    """
    Create and display a folium map from coordinates based on German Web Map Services that includes service station and car/truck parking polygons

    Parameters:
    coord (list): [lat, lon] Coordinates.
    service_stations (geopandas df): Geopandas Dataframe of service stations containing service station area polygons.
    parking (geopandas df): Geopandas Dataframe containing truck and car parking spaces
    zoom_start (int): Zoom start level for Folium map.
    max_zoom (int): Maximum zooom level for Folium map.
    """

    # Instantiate folium map
    m = folium.Map(location=coord, zoom_start=zoom_start, max_zoom = max_zoom)

    def add_wms_layer(layer_name, wms_url, folium_base_map):
        """
        Add a WMS layer to a Folium map.

        Parameters:
        layer_name (str): The name assigned to layer.
        wms_url (str): the GetCapabilities url of the WMS layer.
        folium_base_map (folium.Map): The base map item.
        """

        # Define the WMS layer parameters
        wms_layer = folium.raster_layers.WmsTileLayer(
            url=wms_url,
            layers=layer_name,
            name=layer_name,
            fmt='image/png',
            transparent=True,
            overlay=True,
            control=False,
        )

        wms_layer.add_to(folium_base_map)

    # Define layers and WMS layer urls as dictionary
    wms_urls = {
        'WMS_NI_DOP': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS', # Lower Saxony
        'bebb_dop20c': 'https://isk.geobasis-bb.de/mapproxy/dop20c/service/wms?', # Brandenburg & Berlin
        'th_dop': 'https://www.geoproxy.geoportal-th.de/geoproxy/services/DOP?version=1.1.1&', # Thuringia
        'lsa_lvermgeo_dop20_2': 'https://www.geodatenportal.sachsen-anhalt.de/wss/service/ST_LVermGeo_DOP_WMS_OpenData/guest?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetLegendGraphic&LAYER=lsa_lvermgeo_dop20_2&FORMAT=image/png', # Saxony-Anhalt
        'he_dop_rgb': 'https://gds-srv.hessen.de/cgi-bin/lika-services/ogc-free-images.ows?', # Hessia
        'sh_dop20_rgb': 'https://dienste.gdi-sh.de/WMS_SH_DOP20col_OpenGBD?' # Schleswig-Holstein
    }

    # Add all WMS layers to basemap
    for layer, url in wms_urls.items():
        add_wms_layer(layer, url, m)

    # Add service station are polygon to folium map in yellow
    for index, row in service_stations.iterrows():
        # Ensure that the geometry exists
        if row.geometry:
            # Convert the geometry to GeoJSON
            geo_json = folium.GeoJson(row.geometry.__geo_interface__,
                                      style_function=lambda feature: {
                                          "color": "yellow"
                                      },
            ).add_to(m)

    # Add truck and car parking polygons to folium map in green and blue
    for index, row in parking.iterrows():
        # Ensure that the geometry exists
        if row.geometry:
            # Convert the geometry to GeoJSON
            if row["type"] == "car":
                geo_json = folium.GeoJson(row.geometry.__geo_interface__,
                                          style_function=lambda feature: {
                                              "color": "green"
                                          },
                ).add_to(m)
            elif row["type"] == "truck":
                geo_json = folium.GeoJson(row.geometry.__geo_interface__,
                                          style_function=lambda feature: {
                                              "color": "blue"
                                          },
                ).add_to(m)
            else:
                raise ValueError("No parking type given.")

    # Create red dot on folium map to indicate the current service area
    lat = coord[0]
    lon = coord[1]
    folium.CircleMarker(location=[lat, lon],
                        radius=2,
                        weight=5,
                        color="red"
                        ).add_to(m)

    # Display the map
    display(m)

In [85]:
def create_coord(polygon):
    """
    Create a bounding box [left, bottom, right, top] from a polygon.

    Parameters:
    polygon: (shapely.geometry.Polygon): The input polygon

    Returns:
    A list containing the bounding box coordinates [left, bottom, right, top].
    """

    # Get the bounding box coordinates
    bottom, left, top, right = polygon.bounds

    # Select the middle point of the bounding box
    lon = (bottom + top) / 2
    lat = (left + right) / 2

    # Return the middle point coordinates as a list
    return [lat, lon]

# Show service stations and save correct service stations to list

## Load GeoJSONs as geopandas dataframes

In [7]:
# Load the GeoJSON files containing service station polygons and truck/car parking polygons together with service station IDs (created in 01a_OSM_rest_stops.ipynb)
all_parkings = gpd.read_file("/content/drive/MyDrive/Master Thesis/01 Data Acquisition/OSM GeoJSON/all_parkings.geojson") # polygons of car and truck parking space
rest_stations = gpd.read_file("/content/drive/MyDrive/Master Thesis/01 Data Acquisition/OSM GeoJSON/rest_stations.geojson") # polygons and ids of rest stations

## Inspect geopandas dataframes

In [89]:
# Inspect all_parkings
all_parkings.head()

,id_car,@id,name,id_rest,type,geometry
0,relation/13277622,relation/13277622,None,lon_10.2703215_lat_53.3301407,car,"MULTIPOLYGON (((10.26903 53.33039, 10.26926 53..."
1,relation/13277623,relation/13277623,None,lon_10.2712414_lat_53.331074,car,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53..."
2,relation/15482166,relation/15482166,None,lon_12.140788_lat_52.2431929,car,"MULTIPOLYGON (((12.14216 52.24271, 12.14213 52..."
3,relation/15482306,relation/15482306,None,lon_12.149456_lat_52.2449316,car,"MULTIPOLYGON (((12.14713 52.24497, 12.14717 52..."
4,way/31129920,way/31129920,Nord-Ostsee-Kanal,lon_9.323161_lat_54.0686721,car,"POLYGON ((9.32127 54.06898, 9.32314 54.06861, ..."


In [90]:
# Inspect rest_stations
rest_stations.head()

,id,@id,highway,name,geometry
0,lon_11.2442867_lat_51.3248863,relation/3947321,rest_area,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,lon_13.5458137_lat_52.7129474,relation/4672225,rest_area,Probstheide,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,lon_13.549042_lat_52.7092396,relation/4672228,rest_area,Ladeburger Heide,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,lon_8.6630029_lat_50.2542348,way/22568867,rest_area,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
4,lon_8.6624068_lat_50.2544468,way/22568868,rest_area,Spießwald,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."


In [93]:
# Inspect dimensions of the geopandas dataframes
print("all_parkings contains", all_parkings.shape[0], "unique car and truck parking spaces.")
print("rest_stations contains", rest_stations.shape[0], "unique rest stations.")

all_parkings contains 1606 unique car and truck parking spaces.
rest_stations contains 506 unique rest stations.


## Select correctly labelled service stations

In [ ]:
# For every service station in rest_stations, create and show a folium map and input whether the service station and the car/truck parking areas are correctly labelled.

# Create empty lists and set index_counter to zero
index_counter = 0

service_stations_correct = []
service_stations_wrong = []

# Loop over all the service stations, shos corresponding folium map and input whether annotation is accurate
for index, row in rest_stations.iterrows():
    coord = create_coord(row[-1]) # create coordinates from service station polygon
    name = row["name"]
    id = row["id"]

    # Store folium map in a variable
    folium_map = create_folium_map(coord, service_stations=rest_stations, parking=all_parkings)

    # Give map time to render and not cover input box
    time.sleep(1)
    print("\n")

    # Clear the previous output and display the new map
    display(folium_map)
    clear_output(wait=True)

    # Ask user to input whether the service station and car/truck parking annotations are accurate
    user_input = input(f"Are the parking spaces at {name} (id: {id}) correctly labelled? Yes: \"y\". No: \"n\". Exit: \"exit\"")

    if user_input == "y":
        service_stations_correct.append(row["id"])
    elif user_input == "n":
        service_stations_wrong.append(row["id"])
    elif user_input == "exit":
        print(f"Next station to be verified: {index_counter}")
        break
    else:
        print("Invalid user_input. Please enter 'y', 'n', or 'exit'.")

    # Delete the map object to free up memory
    del folium_map

    index_counter += 1

    # Show user lastly checked service station
    print("Last image checked:", row["id"], index_counter - 1)

Are the parking spaces at Rastplatz Bissendorf Süd (id: lon_9.7695159_lat_52.5102873) correctly labelled? Yes: "y". No: "n". Exit: "exit"y
Last image checked: lon_9.7695159_lat_52.5102873 505


In [ ]:
# Show list of correctly annotated service stations

service_stations_correct

['lon_8.6630029_lat_50.2542348',
 'lon_11.3531491_lat_50.9378362',
 'lon_9.6300119_lat_53.8303266',
 'lon_9.6297338_lat_53.8301666',
 'lon_11.0102562_lat_50.8353109',
 'lon_12.1106921_lat_51.4564157',
 'lon_12.1076965_lat_51.4562754',
 'lon_10.1688328_lat_53.3457924',
 'lon_12.0315302_lat_51.2103528',
 'lon_10.7298729_lat_50.5605634',
 'lon_8.7976979_lat_50.4392699',
 'lon_8.7960067_lat_50.4414072',
 'lon_9.0037966_lat_50.1997276',
 'lon_9.006114_lat_50.1972118',
 'lon_10.16857_lat_53.3461669',
 'lon_10.444298_lat_50.9932399',
 'lon_10.4367549_lat_50.9933128',
 'lon_10.6579729_lat_51.9212477',
 'lon_9.9309273_lat_54.1889457',
 'lon_10.0774066_lat_53.2322212',
 'lon_10.0214344_lat_54.2446698',
 'lon_8.4094366_lat_50.0433047',
 'lon_8.409903_lat_50.0435443',
 'lon_12.484777_lat_51.9700491',
 'lon_12.6318286_lat_52.0445002',
 'lon_9.6950885_lat_52.4313338',
 'lon_10.6158177_lat_53.5536922',
 'lon_10.6284325_lat_53.5509146',
 'lon_9.43935_lat_54.6368415',
 'lon_13.8513592_lat_51.4570059',


In [ ]:
# Show list of wrongly annotated service stations

service_stations_wrong

['lon_11.2442867_lat_51.3248863',
 'lon_13.5458137_lat_52.7129474',
 'lon_13.549042_lat_52.7092396',
 'lon_8.6624068_lat_50.2544468',
 'lon_8.6184146_lat_49.7896285',
 'lon_9.7581375_lat_52.4268193',
 'lon_10.2712414_lat_53.331074',
 'lon_8.633307_lat_50.0540941',
 'lon_9.2506723_lat_54.1383328',
 'lon_10.2703215_lat_53.3301407',
 'lon_8.9264579_lat_50.3435768',
 'lon_8.9282225_lat_50.3458508',
 'lon_9.930895_lat_52.2392839',
 'lon_9.9290741_lat_52.2397704',
 'lon_9.9118833_lat_54.3141951',
 'lon_9.9181171_lat_54.315654',
 'lon_14.1236975_lat_51.7547827',
 'lon_10.1741282_lat_51.4096944',
 'lon_10.7778756_lat_51.4723257',
 'lon_11.7243104_lat_51.3999369',
 'lon_12.2147233_lat_51.2415014',
 'lon_9.6690837_lat_51.3960842',
 'lon_9.5648832_lat_51.3206232',
 'lon_9.7765986_lat_51.39495',
 'lon_9.7800847_lat_51.3955138',
 'lon_9.4159049_lat_51.2726438',
 'lon_9.4159061_lat_51.2721518',
 'lon_9.3682659_lat_51.2848607',
 'lon_11.0524687_lat_52.2204778',
 'lon_9.2941255_lat_51.3122625',
 'lon_

In [ ]:
# Print number of correctly and wrongly labelled service stations

print("Number of correctly labelled service stations:", len(service_stations_correct))
print("Number of wrongly labelled service stations:", len(service_stations_wrong))

Number of correctly labelled service stations: 267
Number of wrongly labelled service stations: 239


## Save lists as CSVs

In [ ]:
def save_list_to_csv(list_input, name, file_path):
    """
    Saves list to CSV.

    Parameters:
    list_input (list): The list input.
    name (str): The name assigned to the list.
    file_path (str): The path where the csv is to be stored.

    Returns:
    string: Saving confirmation.
    """

    dictionary = {"id_rest": list_input}
    df = pd.DataFrame(dictionary)
    df.to_csv(f"{file_path}/{name}.csv", index=False)

    return f"{name} saved in {file_path}/{name}.csv."

In [ ]:
# Save lists as CSVs

save_list_to_csv(service_stations_correct, "verified_service_stations", "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/")
save_list_to_csv(service_stations_wrong, "wrongly_labelled_service_stations", "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/")

'wrongly_labelled_service_stations saved in /content/drive/MyDrive/Master Thesis/01 Data Acquisition//wrongly_labelled_service_stations.csv.'